In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf

1. Function to split metadata into train, validation and test sets [DONE]
2. Fucntion to write files from a split into required directury hierarchy [PENDING]

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
def create_split(df: pd.DataFrame, n_folds: int, test_ratio: float) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Splits a DataFrame with two columns (file path, target class) into stratified train, validation, and test sets.

    :param df (pd.DataFrame): Input DataFrame with [file_path, target_class] columns
    :param val_ratio (float): Proportion of data for validation
    :param test_ratio (float): Proportion of data for testing 
    :returns: tuple (train_df, val_df, test_df) as stratified splits
    """
    assert df.shape[1] == 2, "Dataframe should have 2 columns"
    assert 0 < n_folds and 0 < test_ratio < 1
    path = df.iloc[:, 0]
    target = df.iloc[:, -1]

    # defining splits
    path_train, path_test, target_train, target_test = train_test_split(path, target, test_size = test_ratio, random_state=42, stratify=target)
    
    train_df = pd.concat([path_train, target_train], axis=1)

    # concatenating path and target data
    folds = cv_split(train_df, n_folds)
    test_df = pd.concat([path_test, target_test], axis=1)
    
    unique_classes = target.unique()
    

    return folds, test_df, unique_classes
    
def cv_split (df: pd.DataFrame, n_folds) -> list:
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    folds = []
    for fold_index, fold in enumerate(skf.split(df, df.iloc[:, -1])):
        foldtoapppend = df.iloc[fold[-1]]
        folds.append(foldtoapppend)
    return folds

# Let's rename everything

In [17]:
path_destination = '../Project/Data/rare_species 1/splits'
origin_path = '../Project/Data/rare_species 1/images'

def load_image (path: str,image_path: str) -> tf.Tensor:
    print(f'file_path: {path + "/" + image_path}')
    image = tf.io.read_file(path + '/' + image_path) # get the image 
    image = tf.image.decode_jpeg(image, channels=3)
    return image

def save_image (path: str, image_path: str, image: tf.Tensor) -> None:
    image = tf.image.encode_jpeg(image)
    tf.io.write_file(path + '/' + image_path, image)
    
def folder_rearrangement (df: pd.DataFrame, path_destination: str, origin_path: str,labels: list) -> None:
    """
    Rearranges the images in the destination folder according to the DataFrame.

    :param df (pd.DataFrame): DataFrame with [file_path, target_class] columns
    :param path_destination (str): Path to the destination folder
    :param origin_path (str): Path to the original folder
    """
    if not os.path.exists(path_destination):
        os.makedirs(path_destination)
    
    for label in labels:
        if not os.path.exists(path_destination + '/' + label):
            os.makedirs(path_destination + '/' + label)
    for index, im in enumerate(df[df.iloc[:, 1] == label].iloc[:, 0]):
        image = load_image(origin_path, im)
        name_of_image = label + '_' + str(index).zfill(6) + '.jpg'
        final_path_of_image = path_destination + '/' + label + '/' + name_of_image
        save_image(path_destination, name_of_image, image)
        

In [18]:
# Global definitions
path = '../Project/Data/rare_species 1'
import pandas as pd
df = pd.read_csv(path + '/metadata.csv')
df.head()

df = df[['file_path', 'phylum']]


In [19]:
new_path_imgs = '/Users/leonardodicaterina/Desktop/NovaIMS/DL/data'
df_reduced = df.iloc[:, :100]

list_of_folds, test_df, unique_classes = create_split(df, n_folds=5, test_ratio=0.2)
print(f"unique_classes {unique_classes}")
for i, fold in enumerate(list_of_folds):
    path_fold = new_path_imgs + '/fold_' + str(i)
    folder_rearrangement( fold, path_fold, fold, unique_classes)



unique_classes ['mollusca' 'chordata' 'arthropoda' 'echinodermata' 'cnidaria']
file_path:                                                file_path  \
5181   chordata_elapidae/22401524_1055803_eol-full-si...   
4459   chordata_callorhinchidae/29819269_46561158_eol...   
5422   chordata_testudinidae/29620713_794282_eol-full...   
6680   chordata_salamandridae/8936926_4357897_eol-ful...   
9319   chordata_balaenidae/20140543_46559421_eol-full...   
...                                                  ...   
10064  chordata_hemiscylliidae/29599804_46559713_eol-...   
6213   chordata_dasyatidae/29883670_46560871_eol-full...   
9486   chordata_callitrichidae/28288244_323904_eol-fu...   
8734   chordata_alopiidae/22310552_46559744_eol-full-...   
10295  chordata_plethodontidae/20488656_336169_eol-fu...   

                                                  phylum  
5181   chordata/cnidaria_agariciidae/28307534_4527651...  
4459   chordata/cnidaria_agariciidae/28307534_4527651...  
5422   chord

InvalidArgumentError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input filename tensor must be scalar, but had shape: [1918,2] [Op:ReadFile]